![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Advanced Futures Trading Strategies

"Advanced Futures Trading Strategies" by Robert Carver lays the foundation for building a systematic trading system.

Although the book focuses on futures trading, its concepts are foundational and applicable to other financial instruments.

This notebook replicates the book's initial strategies, emphasizing data processing, metric building, and position size management with risk targeting.

# Data aquisition

In [3]:
from datetime import datetime
from QuantConnect import Resolution
from QuantConnect.Securities import Futures
from QuantConnect.Data.UniverseSelection import *
from helper import calculate_and_merge_returns, summarize_statistics, calculate_contract_risk, simulate_buy_and_hold, calculate_rolling_costs
import numpy as np
from scipy.stats import skew

In [42]:
# instruments = {
#     'ES': {
#         'multiplier': 50,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.25,
#         'commission': 2.50,
#         'expiration_dates': [3, 6, 9, 12]
#     },
#     'MES': {
#         'multiplier': 5,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 1.25,
#         'spread': 0.25,
#         'commission': 2.50,
#         'expiration_dates': [3, 6, 9, 12]
#     },
#     'ZN': {  # 10-Year Treasury
#         'multiplier': 1000,
#         'tick_value': 0.015625,
#         'minimum_fluctuation': 15.625,
#         'spread': 0.015625,
#         'commission': 2.50,
#         'expiration_dates': [6, 9, 12]  # Placeholder
#     },
#     'ZF': {  # 5-Year Treasury
#         'multiplier': 1000,
#         'tick_value': 0.0078125,
#         'minimum_fluctuation': 7.8125,
#         'spread': 0.0078125,
#         'commission': 2.50,
#         'expiration_dates': [6, 9, 12]  # Placeholder
#     },
#     'ZB': {  # 30-Year Treasury
#         'multiplier': 1000,
#         'tick_value': 0.015625,
#         'minimum_fluctuation': 15.625,
#         'spread': 0.015625,
#         'commission': 2.50,
#         'expiration_dates': [6, 9, 12]  # Placeholder
#     },
#     'ZW': {  # Wheat
#         'multiplier': 50,
#         'tick_value': 0.0025,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.0025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'CL': {  # Crude Oil
#         'multiplier': 1000,
#         'tick_value': 0.01,
#         'minimum_fluctuation': 10.00,
#         'spread': 0.01,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'GC': {  # Gold
#         'multiplier': 100,
#         'tick_value': 0.1,
#         'minimum_fluctuation': 10.00,
#         'spread': 0.1,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'HE': {  # Lean Hog
#         'multiplier': 400,
#         'tick_value': 0.00025,
#         'minimum_fluctuation': 10.00,
#         'spread': 0.00025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'KC': {  # Coffee
#         'multiplier': 375,
#         'tick_value': 0.0005,
#         'minimum_fluctuation': 18.75,
#         'spread': 0.0005,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#         'SB': {  # Sugar
#         'multiplier': 1120,
#         'tick_value': 0.0001,
#         'minimum_fluctuation': 11.20,
#         'spread': 0.0001,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'GF': {  # Feeder Cattle
#         'multiplier': 500,
#         'tick_value': 0.00025,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.00025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'ZS': {  # Soy Bean
#         'multiplier': 50,
#         'tick_value': 0.0025,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.0025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'ZC': {  # Corn
#         'multiplier': 50,
#         'tick_value': 0.0025,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.0025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'NG': {  # Natural Gas
#         'multiplier': 10000,
#         'tick_value': 0.001,
#         'minimum_fluctuation': 10.00,
#         'spread': 0.001,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'SI': {  # Silver
#         'multiplier': 5000,
#         'tick_value': 0.005,
#         'minimum_fluctuation': 25.00,
#         'spread': 0.005,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'HG': {  # Copper
#         'multiplier': 25000,
#         'tick_value': 0.0005,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.0005,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'NQ': {  # Nasdaq
#         'multiplier': 20,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 5.00,
#         'spread': 0.25,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'CT': {  # Cotton
#         'multiplier': 500,
#         'tick_value': 0.0001,
#         'minimum_fluctuation': 5.00,
#         'spread': 0.0001,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     }
# }



In [30]:
# Define Instruments Dictionary
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.SP500EMini,
        'resolution': Resolution.Daily
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.MicroSP500EMini,
        'resolution': Resolution.Daily
    },
    'USDMXN': {
        'subscription_type': 'Forex',
        'subscription_details': 'USDMXN',
        'resolution': Resolution.Daily,
        'multiplier': 1,

    }
}



In [11]:


# Initialize QuantBook
qb = QuantBook()

# Add Subscriptions for Futures and Forex
subscriptions = {}
for symbol, details in instruments.items():
    if details['subscription_type'] == 'Future':
        future = qb.AddFuture(details['subscription_details'], details['resolution'])
        subscriptions[symbol] = future.Symbol
    elif details['subscription_type'] == 'Forex':
        forex = qb.AddForex(details['subscription_details'], details['resolution'])
        subscriptions[symbol] = forex.Symbol

# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()
trading_days_per_year = 256
capital = 100000
target_risk = .20

# Request Historical Data
timeseries = {}
for symbol, sub_symbol in subscriptions.items():
    history = qb.History([sub_symbol], start_date, end_date, instruments[symbol]['resolution'])
    if instruments[symbol]['subscription_type'] == 'Future':
        history.reset_index(level=0, drop=True, inplace=True)
    timeseries[symbol] = history[['close']]

del details
del end_date
del forex
del future
del history
del start_date
del sub_symbol
del symbol


## Print the timeseries

In [21]:
#Print symbols and first 5 entires of historical_data
for symbol, data in timeseries.items():
    print(symbol)
    print(data.head())
    # print index format
    print(data.index)

ES
                                  close  price_return  currency_return  \
symbol time                                                              
ES 1S1 2007-01-04 19:00:00  1383.106300      2.905177       145.258845   
       2007-01-05 19:00:00  1371.727690    -11.378610      -568.930475   
       2007-01-08 19:00:00  1378.022240      6.294550       314.727497   
       2007-01-09 19:00:00  1376.327554     -1.694687       -84.734326   
       2007-01-10 19:00:00  1381.169515      4.841961       242.098075   

                            percentage_return  
symbol time                                    
ES 1S1 2007-01-04 19:00:00           0.210489  
       2007-01-05 19:00:00          -0.822685  
       2007-01-08 19:00:00           0.458878  
       2007-01-09 19:00:00          -0.122980  
       2007-01-10 19:00:00           0.351803  
MultiIndex([('ES 1S1', '2007-01-04 19:00:00'),
            ('ES 1S1', '2007-01-05 19:00:00'),
            ('ES 1S1', '2007-01-08 19:00:00'),
 

## Individual dataframe creation

In [29]:

for symbol, data in timeseries.items():
    globals()[symbol] = pd.DataFrame(data)


## Summary statistics
We can begin working with the aquired data to calculate some useful statistics, such as returns, standard deviations, skew and fat tail ratios. Use the helper function to process these statistics.

### Returns Time Series

In [13]:

# Computes returns in price points, notional value, and percentage
for symbol, price_df in timeseries.items():
    price_series = price_df['close']
    merged_df = calculate_and_merge_returns(symbol, price_series, instruments)
    timeseries[symbol] = merged_df.loc[:, ['close', 'price_return', 'currency_return', 'percentage_return']]

del price_df, price_series, merged_df


### Summary Stats for each symbol

In [14]:
summary = summarize_statistics(timeseries, trading_days_per_year)


Summary Statistics for Each Symbol:


In [15]:
print("Summary Statistics for Each Symbol:")

for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(summary[symbol])


Summary Statistics for Each Symbol:
ES:
{'mean_return': 0.037861581716606904, 'std_dev': 1.2503598362413486, 'annualized_mean': 9.692564919451367, 'annualized_std_dev': 20.005757379861578, 'daily_sharpe_ratio': 0.030280548542266784, 'annualized_sharpe_ratio': 0.48448877667626855, 'monthly_skew': -14.351501476816743, 'lower_fat_tail_ratio': 2.566152652495252, 'higher_fat_tail_ratio': 1.7476788394191025}
MES:
{'mean_return': 0.05079779483730788, 'std_dev': 1.3014846844242447, 'annualized_mean': 13.004235478350818, 'annualized_std_dev': 20.823754950787915, 'daily_sharpe_ratio': 0.039030651259472936, 'annualized_sharpe_ratio': 0.624490420151567, 'monthly_skew': -7.616867247540764, 'lower_fat_tail_ratio': 2.1262043532241814, 'higher_fat_tail_ratio': 1.42063709601869}
USDMXN:
{'mean_return': 0.010732769178629475, 'std_dev': 0.7346441980411229, 'annualized_mean': 2.7475889097291457, 'annualized_std_dev': 11.754307168657967, 'daily_sharpe_ratio': 0.01460947926526562, 'annualized_sharpe_ratio':

## Backtest buy and hold 1 contract. Return is a dataframe or time series.

In [55]:
# Added to helper.py

def calculate_rolling_costs(instrument_details, timeseries):
    if 'rolling_months' not in instrument_details:
        return 0  # No rolling months, hence no rolling costs
    
    rolling_costs = 0
    rolling_months = instrument_details['rolling_months']
    years = range(timeseries.index[0][1].year, timeseries.index[-1][1].year + 1)
    
    for year in years:
        for month in rolling_months:
            # Find the first available trading day in the rolling month
            roll_date = pd.Timestamp(year=year, month=month, day=1)
            found = False
            while roll_date.month == month:
                if roll_date.date() in timeseries.index.get_level_values(1).date:
                    cost_per_roll = instrument_details['commission'] + (instrument_details['spread'] / 2) * instrument_details['multiplier']
                    rolling_costs += cost_per_roll * 2  # Buy and Sell
                    found = True
                    break
                roll_date += pd.Timedelta(days=1)
            if not found:
                print(f"No trading day found in month: {month} of year: {year}")
    
    return rolling_costs



In [56]:
# # Rolling costs per instrument. uncomment to compute

# rolling_costs_per_instrument = {}

# for symbol, data in timeseries.items():
#     instrument_details = instruments[symbol]
#     rolling_costs = calculate_rolling_costs(instrument_details, data)
#     rolling_costs_per_instrument[symbol] = rolling_costs

# print(rolling_costs_per_instrument)


No trading day found in month: 6 of year: 2024
No trading day found in month: 9 of year: 2024
No trading day found in month: 12 of year: 2024
No trading day found in month: 3 of year: 2019
No trading day found in month: 6 of year: 2024
No trading day found in month: 9 of year: 2024
No trading day found in month: 12 of year: 2024
{'ES': 1207.5, 'MES': 125.0, 'USDMXN': 0}


In [58]:
# Added to helper.py
def compute_fx_change(returns, timeseries):
    fx_rate = timeseries['close'].iloc[-1]  # Extract the last closing price of USDMXN
    converted_returns = returns * fx_rate
    return converted_returns

In [60]:
# Added to helper.py
def simulate_buy_and_hold(timeseries, instrument_details, fx_timeseries, num_contracts=1):
    """
    Function to simulate buy and hold strategy for a futures contract

    inputs:
    timeseries: pandas DataFrame containing the price data
    instrument_details: dictionary containing the instrument details
    fx_timeseries: pandas DataFrame containing the forex data for currency conversion
    num_contracts: number of contracts to trade

    outputs:
    net_return_base: net return in base currency
    net_return_mxn: net return in MXN
    """
    first_price = timeseries['close'].iloc[0]
    last_price = timeseries['close'].iloc[-1]

    # Calculate rolling costs
    rolling_costs = calculate_rolling_costs(instrument_details, timeseries)
    
    # Calculate net return in base currency
    net_return_base = (last_price - first_price) * instrument_details['multiplier'] * num_contracts - rolling_costs
    
    # Convert net return to MXN
    net_return_mxn = compute_fx_change(net_return_base, fx_timeseries)
    
    return net_return_base, net_return_mxn

In [62]:
buyhold_results = {}

for symbol, data in timeseries.items():
    print(f"Symbol: {symbol}, Start Date: {data.index[0][1]}, End Date: {data.index[-1][1]}")
    instrument_details = instruments[symbol]
    fx_data = timeseries['USDMXN']
    num_contracts = 1  # Specify the number of contracts as needed
    
    bnh_return, bnh_return_mxn = simulate_buy_and_hold(data, instrument_details, fx_data, num_contracts)
    buyhold_results[symbol] = {
        'bnh_return': bnh_return,
        'bnh_return_mxn': bnh_return_mxn
    }
print(buyhold_results)



Symbol: ES, Start Date: 2007-01-04 19:00:00, End Date: 2024-05-24 20:00:00
No trading day found in month: 6 of year: 2024
No trading day found in month: 9 of year: 2024
No trading day found in month: 12 of year: 2024
Symbol: MES, Start Date: 2019-05-07 20:00:00, End Date: 2024-05-24 20:00:00
No trading day found in month: 3 of year: 2019
No trading day found in month: 6 of year: 2024
No trading day found in month: 9 of year: 2024
No trading day found in month: 12 of year: 2024
Symbol: USDMXN, Start Date: 2007-01-02 19:00:00, End Date: 2024-05-24 20:00:00
{'ES': {'bnh_return': 193962.18500659996, 'bnh_return_mxn': 3245930.0113795497}, 'MES': {'bnh_return': 11185.416382574163, 'bnh_return_mxn': 187186.3772040851}, 'USDMXN': {'bnh_return': 5.96386, 'bnh_return_mxn': 99.80436215960002}}


# Strategy 2: Risk measurement, risk target and capital


## Contract risk in base currency

Update summary dictionary to include the daily and annual contract risk 

risk in currency terms = notional exposure * std dev

notional exposure = last closing price * multiplier

In [50]:
# Update summary with contract risks

summary = calculate_contract_risk(summary, instruments, timeseries)
print("Summary Statistics for Each Symbol with Contract Risks:")
for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(summary[symbol])

In [51]:
target_risk = .20

# Compute target risk in currency
target_risk_in_usd = capital * target_risk


In [52]:
# Compute position size
def calculate_position_size(target_risk_in_usd, contract_risk, capital):
    return target_risk_in_usd / contract_risk

In [53]:
# Calculate position size for each symbol
position_sizes = {}
for symbol, stats in summary.items():
    position_size = calculate_position_size(target_risk_in_usd, stats['annualized_contract_risk'], capital)
    position_sizes[symbol] = position_size
# Print position sizes
print ("Position sizes to achieve target risk per instrument:")
